In [ ]:
pip install -U "jina[standard]"

     |████████████████████████████████| 292 kB 7.1 MB/s 
     |████████████████████████████████| 636 kB 12.9 MB/s 
     |████████████████████████████████| 3.8 MB 24.2 MB/s 
     |████████████████████████████████| 146 kB 61.0 MB/s 
     |████████████████████████████████| 103 kB 55.7 MB/s 
     |████████████████████████████████| 208 kB 58.9 MB/s 
     |████████████████████████████████| 3.2 MB 57.9 MB/s 
     |████████████████████████████████| 1.3 MB 47.5 MB/s 
     |████████████████████████████████| 52 kB 926 kB/s 
     |████████████████████████████████| 50 kB 7.8 MB/s 
     |████████████████████████████████| 1.8 MB 65.3 MB/s 
     |████████████████████████████████| 10.1 MB 20.7 MB/s 
     |████████████████████████████████| 54 kB 4.2 MB/s 
     |████████████████████████████████| 344 kB 59.9 MB/s 
     |████████████████████████████████| 142 kB 67.9 MB/s 
     |████████████████████████████████| 294 kB 66.9 MB/s 
     |████████████████████████████████| 68 kB 7.0 MB/s 
     |████████████████

In [ ]:
!pip install jina

In [ ]:
from jina import hello
jina hello --help

SyntaxError: ignored

In [ ]:
import numpy as np
from jina import Document, DocumentArray, Executor, Flow, requests

In [ ]:
class CharEmbed(Executor):  # a simple character embedding with mean-pooling
    offset = 32  # letter `a`
    dim = 127 - offset + 1  # last pos reserved for `UNK`
    char_embd = np.eye(dim) * 1  # one-hot embedding for all chars

    @requests
    def foo(self, docs: DocumentArray, **kwargs):
        for d in docs:
            r_emb = [ord(c) - self.offset if self.offset <= ord(c) <= 127 else (self.dim - 1) for c in d.text]
            d.embedding = self.char_embd[r_emb, :].mean(axis=0)  # mean-pooling

In [ ]:
class Indexer(Executor):
    _docs = DocumentArray()  # for storing all document in memory

    @requests(on='/index')
    def foo(self, docs: DocumentArray, **kwargs):
        self._docs.extend(docs)  # extend stored `docs`

    @requests(on='/search')
    def bar(self, docs: DocumentArray, **kwargs):
        q = np.stack(docs.get_attributes('embedding'))  # get all embedding from query docs
        d = np.stack(self._docs.get_attributes('embedding'))  # get all embedding from stored docs
        euclidean_dist = np.linalg.norm(q[:, None, :] - d[None, :, :], axis=-1)  # pairwise euclidean distance
        for dist, query in zip(euclidean_dist, docs):  # add & sort match
            query.matches = [Document(self._docs[int(idx)], copy=True, scores={'euclid': d}) for idx, d in enumerate(dist)]
            query.matches.sort(key=lambda m: m.scores['euclid'].value)  # sort matches by its value

In [ ]:
def print_matches(req):  # the callback function invoked when task is done
    for idx, d in enumerate(req.docs[0].matches[:3]):  # print top-3 matches
        print(f'[{idx}]{d.scores["euclid"].value:2f}: "{d.text}"')

In [ ]:
f = Flow(port_expose=12345).add(uses=CharEmbed, parallel=2).add(uses=Indexer)  # build a Flow, with 2 parallel CharEmbed, tho unnecessary

source_code = """
import numpy as np
from jina import Document, DocumentArray, Executor, Flow, requests
class CharEmbed(Executor):  # a simple character embedding with mean-pooling
    offset = 32  # letter `a`
    dim = 127 - offset + 1  # last pos reserved for `UNK`
    char_embd = np.eye(dim) * 1  # one-hot embedding for all chars
    @requests
    def foo(self, docs: DocumentArray, **kwargs):
        for d in docs:
            r_emb = [ord(c) - self.offset if self.offset <= ord(c) <= 127 else (self.dim - 1) for c in d.text]
            d.embedding = self.char_embd[r_emb, :].mean(axis=0)  # average pooling
class Indexer(Executor):
    _docs = DocumentArray()  # for storing all documents in memory
    @requests(on='/index')
    def foo(self, docs: DocumentArray, **kwargs):
        self._docs.extend(docs)  # extend stored `docs`
    @requests(on='/search')
    def bar(self, docs: DocumentArray, **kwargs):
        q = np.stack(docs.get_attributes('embedding'))  # get all embeddings from query docs
        d = np.stack(self._docs.get_attributes('embedding'))  # get all embeddings from stored docs
        euclidean_dist = np.linalg.norm(q[:, None, :] - d[None, :, :], axis=-1)  # pairwise euclidean distance
        for dist, query in zip(euclidean_dist, docs):  # add & sort match
            query.matches = [Document(self._docs[int(idx)], copy=True, scores={'euclid': d}) for idx, d in enumerate(dist)]
            query.matches.sort(key=lambda m: m.scores['euclid'].value)  # sort matches by their values
f = Flow(port_expose=12345, protocol='http', cors=True).add(uses=CharEmbed, parallel=2).add(uses=Indexer)  # build a Flow, with 2 parallel CharEmbed, tho unnecessary
with f:
    f.post('/index', DocumentArray([Document(text=t.strip()) for t in source_code.split('\n') if t.strip() ])) # index all lines of this notebook's source code
    f.post('/search', Document(text='@request(on=something)'), on_done=print_matches)
"""

with f:
    f.post('/index', DocumentArray([Document(text=t.strip()) for t in source_code.split('\n') if t.strip() ])) # index all lines of this notebook's source code
    f.post('/search', Document(text='@request(on=something)'), on_done=print_matches)


      pod0/head@62[L]:ready and listening
     pod0/pea-0@62[L]:ready and listening
     pod0/pea-1@62[L]:ready and listening
      pod0/tail@62[L]:ready and listening
           pod1@62[L]:ready and listening
        gateway@62[L]:ready and listening
           Flow@62[I]:🎉 Flow is ready to use!
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:12345
	🔒 Private network:	172.28.0.2:12345
	🌐 Public address:	34.125.118.210
:12345
[0]0.123462: "f.post('/search', Document(text='@request(on=something)'), on_done=print_matches)"
[1]0.157459: "@requests(on='/index')"
[2]0.171835: "@requests(on='/search')"


In [ ]:
import IPython
IPython.display.HTML(filename='/path/to/your/filename')